In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
'''df1 = pd.read_csv('spring-framework.txt', sep="\t", header=None)
df1.columns = ["RevNo", "Rev", "Message", "Generated"]

df2 = pd.read_csv('spring-boot.txt', sep="\t", header=None)
df2.columns = ["RevNo", "Rev", "Message", "Generated"]

df3 = pd.read_csv('elasticsearch.txt', sep="\t", header=None)
df3.columns = ["RevNo", "Rev", "Message", "Generated"]

df4 = pd.read_csv('hadoop.txt', sep="\t", header=None)
df4.columns = ["RevNo", "Rev", "Message", "Generated"]

df5 = pd.read_csv('mockito.txt', sep="\t", header=None)
df5.columns = ["RevNo", "Rev", "Message", "Generated"]

data = pd.concat([df1, df2, df3, df4, df5], axis=0)'''

'df1 = pd.read_csv(\'spring-framework.txt\', sep="\t", header=None)\ndf1.columns = ["RevNo", "Rev", "Message", "Generated"]\n\ndf2 = pd.read_csv(\'spring-boot.txt\', sep="\t", header=None)\ndf2.columns = ["RevNo", "Rev", "Message", "Generated"]\n\ndf3 = pd.read_csv(\'elasticsearch.txt\', sep="\t", header=None)\ndf3.columns = ["RevNo", "Rev", "Message", "Generated"]\n\ndf4 = pd.read_csv(\'hadoop.txt\', sep="\t", header=None)\ndf4.columns = ["RevNo", "Rev", "Message", "Generated"]\n\ndf5 = pd.read_csv(\'mockito.txt\', sep="\t", header=None)\ndf5.columns = ["RevNo", "Rev", "Message", "Generated"]\n\ndata = pd.concat([df1, df2, df3, df4, df5], axis=0)'

In [19]:
data = pd.read_csv('slf4j/slf4j.txt', sep='\t', header=None)
data.columns = ["RevNo", "PrevRev", "Rev", "Message", "Generated"]

print(len(data))

#remove empty string
data = data[data['Generated'] != '']
data = data[data['Message'] != '']

#remove NaN values
data = data[pd.notnull(data['Generated'])]
data = data[pd.notnull(data['Message'])]

#Remove merge commit and 'discarded' generated message
data.loc[:,'Message'] = data['Message'].apply(lambda x: '' if x.startswith("Merge") else x)
data.loc[:,'Generated'] = data['Generated'].apply(lambda x: '' if x.startswith("Discarded") else x)

#remove special characters
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))

#remove digits
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: x.translate(None, string.digits))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: x.translate(None, string.digits))

#split camel case words
#data.loc[:,'Message'] = data['Message'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))

#remove small commit messages and long generated messages
data.loc[:,'Message'] = data['Message'].apply(lambda x: '' if len(x.split(" ")) < 3 else x)
data.loc[:,'Generated'] = data['Generated'].apply(lambda x: '' if len(x.split(" ")) > 300 else x)

#again check empty and NaN values
data = data[pd.notnull(data['Generated'])]
data = data[data['Generated'] != '']
data = data[pd.notnull(data['Message'])]
data = data[data['Message'] != '']

#make two new columns 
#data.loc[:,'Ori'] = data['Message'].apply(lambda x: x.lower())
#data.loc[:,'Gen'] = data['Generated'].apply(lambda x: x.lower())

#make tokens of the newly generated columns and store in two addtional columns
data.loc[:,'OriTok'] = data['Message'].apply(lambda x: [x.strip() for x in x.split(" ")])
data.loc[:,'GenTok'] = data['Generated'].apply(lambda x: [x.strip() for x in x.split(" ")])

1567


In [20]:
len(data)

435

In [16]:
oriVocab = set()
genVocab = set()
for index, row in data.iterrows():
    oriVocab.update(row['OriTok'])
    genVocab.update(row['GenTok'])

In [17]:
file1 = file('spring-framework/vocab.ori', 'w')
file2 = file('spring-framework/vocab.gen', 'w')

for item in list(oriVocab):
    if len(item) <= 2:
        continue
    file1.write(item)
    file1.write('\n')
    
for item in list(genVocab):
    if len(item) <= 2:
        continue
    file2.write(item)
    file2.write('\n')

file1.close()
file2.close()

In [22]:
total = len(data)
n = int(total * 0.8)
m = total - n
train_data = data.head(n)
test_data = data.tail(m)

In [23]:
len(train_data)

3313

In [24]:
len(test_data)

829

In [25]:
train_data['Message'].to_csv('spring-framework/train.ori', sep='\n', index=False, header=False)
train_data['Generated'].to_csv('spring-framework/train.gen', sep='\n', index=False, header=False)

In [26]:
test_data['Message'].to_csv('spring-framework/test.ori', sep='\n', index=False, header=False)
test_data['Generated'].to_csv('spring-framework/test.gen', sep='\n', index=False, header=False)

test_data['Message'].to_csv('spring-framework/validation.ori', sep='\n', index=False, header=False)
test_data['Generated'].to_csv('spring-framework/validation.gen', sep='\n', index=False, header=False)